In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

import numpy as np
import pandas as pd
import seaborn as sns

from access_biology_data import annotation, meta
from access_economic_data import patents
from access_literature_data import medline
from access_science_shared import standardizer

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sys.path.append('./../src/')
import nar170605f_funding as nar_funding
import nar170604f_occurences as nar_occurences
import resci_tools as ret

# General reference

In [ ]:
taxon_id = 9606
ref_genes = standardizer.reference_genes(taxon_id, 'orp')
save_images = False
save_images_group2 = False  #added figures
save_tables = True

# Papers

In [ ]:
gene2pubmed = medline.gene2pubmed(taxon_id, ['pubmed_id', 'gene_ncbi'], paper_kind='research')
papers = nar_occurences.count_papers_and_attention(ref_genes, gene2pubmed)

# Patents

In [ ]:
rosenfeld = patents.rosenfeld_2013()
rosenfeld = rosenfeld[rosenfeld['gene_ncbi'].isin(ref_genes)]
genes_per_patent = rosenfeld['patent'].value_counts()

In [ ]:
df = pd.DataFrame(index=ref_genes)

In [ ]:
p = genes_per_patent[genes_per_patent==1].index    # single gene patents
d = rosenfeld[rosenfeld['patent'].isin(p)]
df.loc[:, 'patents (max. 1 gene/patent)'] = d['gene_ncbi'].value_counts()

d = rosenfeld
df.loc[:, 'patents (any amount of genes/patent)'] = d['gene_ncbi'].value_counts()

patents = df.fillna(0)

In [ ]:
# for c in [100000, 1000, 100, 10, 1]:


# supremum_patents = genes_per_patent.max() + 1   # higher than 

# for c in [supremum_patents + 1, 1]:
#     p = genes_per_patent[genes_per_patent<=c].index
#     d = rosenfeld[rosenfeld['patent'].isin(p)]
#     df.loc[:, 'patents (max. {} g.)'.format(c)] = d['gene_ncbi'].value_counts()
# patents = df.fillna(0)

# Synonyms

In [ ]:
def count_words(s):
    if s == '-':
        w = 0
    else:
        c = np.count_nonzero([x == '|' for x in s])
        w = c + 1
    return w

In [ ]:
gene_info = meta.gene_info(taxon_id=taxon_id)
gene_info = gene_info[gene_info['gene_ncbi'].isin(ref_genes)]
gene_info = gene_info.set_index('gene_ncbi', verify_integrity=True)

In [ ]:
od = gene_info['Other_designations'].apply(count_words)
sy = gene_info['Synonyms'].apply(count_words)

In [ ]:
nomina = pd.concat([od, sy], axis=1)
nomina = nomina.rename(columns={'Other_designations': 'names', 'Synonyms': 'symbols'})
nomina = nomina + 1  # add count for NIH symbol and HUGO names
# nomina['names_and_symbols'] = nomina['names'] + nomina['symbols']

# GO

In [ ]:
def get_go(cat, negating, temporary):

    a = annotation.go(
        taxon_id,
        category=[cat],
        negating_support=[negating],
        temporary_evidence=[temporary],
        unmapped_evidence=[False])

    a = a[['gene_ncbi', 'GO_term']].drop_duplicates()
    a = a.groupby('gene_ncbi').size()
    a.name = 'go_' + cat.lower() + '_negating_' + str(negating) + '_temporary_' + str(temporary)
    return a

In [ ]:
def get_go_all(negating, temporary):

    a = annotation.go(
        taxon_id,
        category=['Function', 'Process', 'Component'],
        negating_support=[negating],
        temporary_evidence=[temporary],
        unmapped_evidence=[False])

    a = a[['gene_ncbi', 'GO_term']].drop_duplicates()
    a = a.groupby('gene_ncbi').size()
    a.name = 'go_negating_' + str(negating) + '_temporary_' + str(temporary)
    return a

In [ ]:
go_all = pd.concat([
    get_go_all(negating=False, temporary=False),
    get_go_all(negating=True, temporary=False),
    get_go_all(negating=False, temporary=True),
], axis=1).fillna(0)

In [ ]:
go = pd.concat(
    [get_go(c, negating=False, temporary=False) for c in ['Function', 'Process', 'Component']] +
    [get_go(c, negating=True, temporary=False) for c in ['Function', 'Process', 'Component']] +
    [get_go(c, negating=False, temporary=True) for c in ['Function', 'Process', 'Component']],
    axis=1
).fillna(0)

In [ ]:
g = annotation.go(
    taxon_id,
    ['Function', 'Process', 'Component'],
    negating_support=[False],
    temporary_evidence=[False],
    unmapped_evidence=[False],
    any_negating_support=[False])

In [ ]:
g = g[['gene_ncbi', 'GO_term']].drop_duplicates()



In [ ]:
v = 1/g['GO_term'].value_counts().to_frame('value_per_category')

In [ ]:
g.head(2)

In [ ]:
g = pd.merge(g, v, left_on='GO_term', right_index=True)

In [ ]:
go_specificity=g[['gene_ncbi','value_per_category']].groupby('gene_ncbi').agg(sum)

# Biosystems

In [ ]:
def get_biosystem(cat):
    a = annotation.biosystems(
        taxon_id,
        cat)
    a = a[['gene_ncbi', 'accession']]
    a = a.groupby('gene_ncbi').size()
    a.name = cat
    return a

In [ ]:
biosystems = pd.concat([get_biosystem(x) for x in ['REACTOME', 'KEGG', 'WikiPathways']], axis=1).fillna(0)

# GeneRIFs

In [ ]:
gene_rif = annotation.generif(taxon_id)

In [ ]:
gene_rif.loc[:,'clean'] = gene_rif.loc[:,'GeneRIF text'].str.lower()
gene_rif.loc[:,'clean'] = gene_rif.loc[:,'clean'].str.replace('\(huge navigator\)','') 
gene_rif.loc[:,'clean'] = gene_rif.loc[:,'clean'].str.replace('\.','')

In [ ]:
gene_rif = gene_rif[gene_rif['gene_ncbi'].isin(ref_genes)]
gene_rif = gene_rif[['gene_ncbi', 'clean']]

In [ ]:
# Thought: quantify language: problem: might need a more unbiased approach if it should sound
# convincing / not pushed within a largerger narrative
# gene_rif = gene_rif.drop_duplicates()
# gene_rif = gene_rif[gene_rif['gene_ncbi'].isin(ref_genes)]
# clean_rif.loc[:,'may'] = clean_rif.loc[:,'clean'].str.contains(' may | maybe ')

In [ ]:
gene_rif = gene_rif.drop_duplicates()
gene_rifs = gene_rif['gene_ncbi'].value_counts().to_frame('gene_rifs')

# Combine different knowledge readouts

In [ ]:
master = pd.concat([
    papers, patents, nomina, go, biosystems, gene_rifs, go_specificity, go_all
], axis=1)
master = master.loc[ref_genes, :]
master = master.fillna(0)

lm = master.apply(lambda  x: np.log10(x + 1))        # note that we are not looking at log 10

lm.columns = ['log_{}'.format(x) for x in lm.columns]
main = pd.concat([master, lm], axis=1)
main.columns

# Export data

In [ ]:
main.index.name = 'gene_ncbi'

use_in_paper = [
    'papers',
    'names',
    'symbols',
    'patents (max. 1 gene/patent)',    
    'patents (any amount of genes/patent)',
    'go_negating_False_temporary_False',
    'go_negating_False_temporary_True',
]

out = main[use_in_paper]

if save_tables:
    ret.export_full_frame(
    '170705_relatedness_of_knowledge/data.csv',
    out
    )

# Visualize

In [ ]:
plain_color = 'navajowhite'

In [ ]:
bins = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4]
main['log_papers_bin'] = pd.cut(main['log_papers'].values, bins, include_lowest=True)

In [ ]:
sns.boxplot(
    x='log_papers_bin',
    y='log_gene_rifs',
    data=main, notch=True,
    color=plain_color)

if save_images:
    ret.export_image('170705_relatedness_of_knowledge/gene_rifs.pdf')

In [ ]:
sns.boxplot(
    x='log_papers_bin',
    y='log_go_process_negating_False_temporary_False',
    data=main,
    notch=True,
    color=plain_color)

In [ ]:
sns.boxplot(
    x='log_papers_bin',
    y='log_go_process_negating_False_temporary_False',
    data=main, notch=True,
    color=plain_color)

In [ ]:
sns.boxplot(
    x='log_papers_bin', 
    y='log_go_component_negating_False_temporary_False', 
    data=main, 
    notch=True, 
    color=plain_color)

In [ ]:
sns.boxplot(
    x='log_papers_bin',
    y='log_go_function_negating_False_temporary_False',
    data=main,
    notch=True,
    color=plain_color)

In [ ]:
sns.boxplot(
    x='log_papers_bin',
    y='log_go_negating_False_temporary_True',
    data=main,
    notch=True,
    color=plain_color)
if save_images:
    ret.export_image('170705_relatedness_of_knowledge/GO_temporary.pdf')

In [ ]:
sns.boxplot(
    x='log_papers_bin',
    y='log_go_negating_False_temporary_False',
    data=main,
    notch=True,
    color=plain_color)
if save_images:
    ret.export_image('170705_relatedness_of_knowledge/GO_all.pdf')

In [ ]:
main['has_one_gene_patent'] = main['patents (max. 1 gene/patent)'] > 0
main['has_any_patent'] = main['patents (any amount of genes/patent)'] > 0
main['has_negating_GO'] = main['go_negating_True_temporary_False'] >0 

In [ ]:
sns.barplot(x="log_papers_bin", y="has_negating_GO", data=main, color='sandybrown')
if save_images:
    ret.export_image('170705_relatedness_of_knowledge/fraction_with_negating_go.pdf')

In [ ]:
sns.barplot(x="log_papers_bin", y="has_one_gene_patent", data=main, color='sandybrown')
if save_images:
    ret.export_image('170705_relatedness_of_knowledge/fraction_with_one_gene_patent.pdf')

In [ ]:
sns.barplot(x="log_papers_bin", y="has_any_patent", data=main, color='sandybrown')
if save_images:
    ret.export_image('170705_relatedness_of_knowledge/fraction_with_a_patent.pdf')

### Side note: 
### Check genes with many designations
Result: these refer to (splice) variants ; note to self: which is kind of intriguing, as genes with most named splice variants appears to be largely unstudied

In [ ]:
main[main['log_names']>1.6]

In [ ]:
gene_info.loc[79054,'Other_designations']

In [ ]:
gene_info.loc[148398,'Other_designations']

### End of Side note – continue



In [ ]:
sns.boxplot(x='log_papers_bin', y='log_names', data=main, notch=True, color=plain_color)
if save_images:
    ret.export_image('170705_relatedness_of_knowledge/names.pdf')

In [ ]:
sns.boxplot(x='log_papers_bin', y='log_symbols', data=main, notch=True, color=plain_color)
if save_images:
    ret.export_image('170705_relatedness_of_knowledge/symbols.pdf')

# Cluster features

In [ ]:
master.columns

In [ ]:
cols_to_compare = [
    'papers', 'attention', 'patents (max. 1 gene/patent)', 
    'patents (any amount of genes/patent)', 'names', 'symbols',
    'go_negating_False_temporary_False', 'go_negating_True_temporary_False',
    'go_negating_False_temporary_True',
    'REACTOME', 'KEGG',
    'WikiPathways', 'gene_rifs']
       
#       'go_function_negating_False_temporary_False',
#       'go_process_negating_False_temporary_False',
#       'go_component_negating_False_temporary_False',
#       'go_function_negating_True_temporary_False',
#       'go_process_negating_True_temporary_False',
#       'go_component_negating_True_temporary_False',
#       'go_function_negating_False_temporary_True',
#       'go_process_negating_False_temporary_True',
#       'go_component_negating_False_temporary_True', 


In [ ]:
g = sns.clustermap(master.loc[:, cols_to_compare].corr('spearman'), method='ward', vmin=-1, vmax=1)
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0)
if save_images:
    ret.export_image('170705_relatedness_of_knowledge/clustergram.pdf')

In [ ]:
g = sns.clustermap(master.loc[:, cols_to_compare].corr('spearman'), method='ward', annot=True, vmin=-1, vmax=1)
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0)
if save_images:
    ret.export_image('170705_relatedness_of_knowledge/clustergram_annotated.pdf')

In [ ]:
cols_to_compare = [
    'papers', 'attention', 'patents (max. 1 gene/patent)', 
    'patents (any amount of genes/patent)', 'names', 'symbols',
    'go_negating_False_temporary_False',
    'go_negating_False_temporary_True',
    'REACTOME', 'KEGG',
    'WikiPathways', 'gene_rifs']

In [ ]:
g = sns.clustermap(master.loc[:, cols_to_compare].corr('spearman'), method='ward', vmin=-1, vmax=1)
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0)
if save_images_group2:
    ret.export_image('170705_relatedness_of_knowledge/clustergram_without_negating.pdf')